In [ ]:
# 建立搜索引擎，实现查询成语含义和查询与当前成语押韵的其他成语
# 项目流程
# 开启elasticsearch建索引-->成语生成拼音-->导入elasticsearch中-->查询
# 使用搜索引擎直接查询，考虑成语押韵情况较少，非严格意义上的三押，四押。

# elasticsearch构建索引语句：
# {
#   "settings": {
#     "index": {
#       "creation_date": "1510042090481",
#       "number_of_shards": "5",
#       "number_of_replicas": "1",
#       "uuid": "zz25wpPuSvqxFsLevXAPpA",
#       "version": {
#         "created": "2030599"
#       }
#     }
#   },
#   "mappings": {
#     "chengyu": {
#       "properties": {
#         "name": {
#           "type": "string",
#           "index": "not_analyzed"
#         },
#         "pinyin": {
#           "type": "string",
#           "index": "not_analyzed"
#         },
#         "mean": {
#           "type": "string",
#           "index": "not_analyzed"
#         }
#       }
#     }
#   }
# }

In [1]:
#成语转拼音
#coding:utf-8
from pypinyin import pinyin,lazy_pinyin

file_read = open('1.txt','r',encoding='gb18030')
file_write = open('chengyu_with_pinyin.txt','w',encoding='gb18030')
for line in file_read.readlines():
    line = line.strip()
    line = line.split('\t')
    line_pinyin = lazy_pinyin(line[0])
    line_pinyin = ' '.join(line_pinyin)
    file_write.write(line[0]+'\t'+line_pinyin+'\t'+line[1]+'\n')
file_write.close()
file_read.close()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Tiger\AppData\Local\Temp\jieba.cache
Loading model cost 2.082 seconds.
Prefix dict has been built succesfully.


In [56]:
##成语导入到es
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(['localhost:9200'])

#导入数据
file = open('chengyu_with_pinyin.txt', 'r', encoding='gb18030')

for row in file.readlines():
    row = row.strip()
    str_context = str(row)
    context = str_context.split('\t')
    
    action = {}
    action['name'] = context[0]
    action['pinyin'] = context[1]
    action['mean'] = context[2]

    es.index(index='chengyu', doc_type='chengyu', body=action)
file.close()

In [21]:
# 查询成语及相同韵脚成语
import re
from pypinyin import pinyin,lazy_pinyin
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es = Elasticsearch(['localhost:9200'])

shengmu = ['b','p','m','f','d','t','n','l','g','k','h','j','q','x','z','c','s','zh','ch','sh','r','y','w']

def get_yunjiao(search):    
    yunmu = []
    for i in search:
        for j in shengmu:
            if i.startswith(j):
                s_len = len(j)
                yunmu.append(i[s_len:])
                break
    return yunmu
index = 1
while index <2:
    index += 1
    search_chengyu = input()
    # 转换成拼音
    search_pinyin = lazy_pinyin(search_chengyu)
    #获取查询语句
    search_sql = []
    for i in range(4):
        temp = '.*'+' .*'.join(get_yunjiao(search_pinyin)[i-4:])
        #print(temp)
        search_sql.append(temp)
    #得到查询结果
    results = []
    for i in range(len(search_sql)):
        result = es.search(index='chengyu', doc_type='chengyu', 
                           body={"fields": ["name","mean","pinyin"], "size":20, "query": {"regexp": {"pinyin": str(search_sql[i])}}})
        results.append(result)
    print('search...')
    # all_chengyu = [search_chengyu]
    all_chengyu = []
    for i in range(len(results)):
        print(str(4-i)+'押：')
        for hit in results[i]['hits']['hits']:
            if hit['fields']['name'][0] not in all_chengyu:
                print(hit['fields']['name'][0],':',hit['fields']['mean'][0])
                all_chengyu.append(hit['fields']['name'][0])
    print('finish.')

尔虞我诈
search...
4押：
尔虞我诈 : 见“尔诈我虞”。
予夺生杀 : 《周礼·天官、大宰》的“八柄”中有予、夺、生、诛（责备）等权力。后因以“予夺生杀”泛指帝王掌握的赏罚生死大权。
3押：
2押：
桀贪骜诈 : 凶暴贪婪而又傲慢狡诈。
一毫不差 : 完全相同，没有一点差异。
装疯作傻 : 见“装疯卖傻”。
装痴作傻 : 见“装痴卖傻”。
簸土扬沙 : 指虚张声势。
高楼大厦 : 高大豪华的房屋建筑。
高堂大厦 : 见“高楼大厦”。
一笔抹煞 : 一笔全部抹掉。常喻轻率地全部否定。
拳中掿沙 : 以手握沙，捏合不拢。比喻不团结。
一笔抹杀 : 见“一笔抹煞”。
阴错阳差 : 古代历数术语。明王逵《蠡海集·历数》：“阴错阳差，有十二月，盖六十甲子分为四段，自甲子、己卯、甲午、己酉，各得十五辰。甲子之前三辰，值辛酉、壬戌、癸亥为阴错；己卯之前三辰，值丙子、丁丑、戊寅为阳差；甲午之前三辰，值辛卯、壬辰、癸巳为阴错；己酉之前三辰，值丙午、丁未、戊申为阳差。盖四段中，每段除十二辰，各馀三辰，三四亦得十二辰，是为阴错阳差也。甲子、甲午为阳辰，故有阴错；己卯、己酉为阴辰，故有阳差也。”后用以比喻因各种偶然因素而造成了差错。
1押：
家至户察 : 到每家每户去察看。
马足龙沙 : 指驰骋边疆，扬威域外。龙沙，塞北沙漠地方。
入海算沙 : ①到海底数沙子。喻白费功夫。②又喻用力精勤。
稳打稳扎 : 同“稳扎稳打”。
一片散沙 : 见“一盘散沙”。
一时半霎 : 犹一时半刻。
云泥之差 : 见“云泥之别”。
折戟沉沙 : 断戟沉埋在沙里。形容失败惨重。
装痴卖傻 : 故意装作痴呆的样子。
装疯卖傻 : 假装疯癫痴呆的样子。
不丰不杀 : 不奢不俭，多少合宜。
垂死挣扎 : 形容最后的反抗。多含贬义。
风云叱咤 : 同“叱咤风云”。
捐残去杀 : 谓放弃残暴和屠杀的行为。
狂轰滥炸 : 疯狂地轰炸。
七步八叉 : 相传曹植七步成诗，温庭筠凡八叉手而赋成八韵，后因以“七步八叉”形容才思敏捷。
finish.


In [62]:
str1 = 'ba zi mei jian'
patten = '.*ian$'
if re.match(patten, str1):
    print('yes')

yes


In [5]:
# #获取词典拼音
# from pypinyin import pinyin,lazy_pinyin
# import pypinyin

# ciyu = open('现代汉语词典第六版电子书.txt','r',encoding='gb18030')
# ciyu_with_py = []
# for line in ciyu:
#     line = line.strip()
#     if '【' in line and '】'in line:
#         #提取【..】中的词语
#         st = line.index('【')
#         en = line.index('】')
#         word = line[st+1:en]
# #         print(word)
# #         print(word_py)
#         word_py = lazy_pinyin(word)
#         word_py = ' '.join(word_py)
#         ciyu_with_py.append(word+' '+word_py)
# ciyu.close()

# len(ciyu_with_py)
# #数据导入到es中
# from datetime import datetime
# from elasticsearch import Elasticsearch
# from elasticsearch import helpers
# es = Elasticsearch(['localhost:9200'])

# #导入数据
# for row in ciyu_with_py:
#     row = row.strip()
#     str_context = str(row)
#     action = {}
#     action['context'] = str_context
#     es.index(index='ciyu2', doc_type='ciyu', body=action)